In [18]:
import librosa
import numpy as np
import openl3
import soundfile as sf
import os
import pandas as pd
from pathlib import Path

In [27]:
import os
import numpy as np
import pandas as pd
import librosa
from pathlib import Path

def extract_features(filepath, sr=22050):
    features = {}
    y, sr = librosa.load(filepath, sr=sr, mono=True)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    features['mfcc'] = np.mean(mfcc, axis=1)
    chroma = librosa.feature.chroma_stft(y=y, sr=sr)
    features['chroma'] = np.mean(chroma, axis=1)
    features['tempo'] = float(librosa.feature.rhythm.tempo(y=y, sr=sr, aggregate=np.median))
    features['spectral_centroid'] = float(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr)))
    features['rms'] = float(np.mean(librosa.feature.rms(y=y)))

    return features

root = "Data\\raw"
features_list = []

for dirpath, _, filenames in os.walk(root):
    for f in filenames:
        if f.lower().endswith(".wav"):
            filepath = os.path.join(dirpath, f)
            feats = extract_features(filepath)
            row = {
                "filepath": filepath,
                "name": os.path.basename(filepath),
                "tempo": feats['tempo'],
                "spectral_centroid": feats['spectral_centroid'],
                "rms": feats['rms'],
                **{f"mfcc_{i}": v for i, v in enumerate(feats['mfcc'])},
                **{f"chroma_{i}": v for i, v in enumerate(feats['chroma'])},
            }
            features_list.append(row)

df = pd.DataFrame(features_list)

out_dir = Path("features")
out_dir.mkdir(parents=True, exist_ok=True)   # tylko folder
out_csv = out_dir / "features.csv"           # ścieżka do pliku
df.to_csv(out_csv, index=False)
print(f"Zapisano: {out_csv}")


C:\Users\dell\AppData\Local\Temp\ipykernel_37004\1261233607.py:20: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  features['tempo'] = float(librosa.feature.rhythm.tempo(y=y, sr=sr, aggregate=np.median))


Zapisano: features\features.csv


In [31]:
import pandas as pd
from pathlib import Path

path = Path("features/features.csv")
df = pd.read_csv(path)

try:
    from IPython import get_ipython
    if get_ipython() is not None:
        from IPython.display import display
        display(df)                     # pokaż całą tabelę w notatniku
    else:
        raise RuntimeError("not notebook")
except Exception:
    # Widok w terminalu: nie ucinaj kolumn/szerokości
    pd.set_option("display.max_columns", None)
    pd.set_option("display.width", None)
    pd.set_option("display.max_rows", 50)
    print(df.to_string(index=False))


,filepath,name,tempo,spectral_centroid,rms,mfcc_0,mfcc_1,mfcc_2,mfcc_3,mfcc_4,...,chroma_2,chroma_3,chroma_4,chroma_5,chroma_6,chroma_7,chroma_8,chroma_9,chroma_10,chroma_11
0,Data\raw\19th Chopin Competition\first round\J...,"Barcarolle in F sharp major, Op. 60.wav",103.359375,636.267132,0.032146,-392.46375,198.91046,13.331565,9.120192,8.214742,...,0.244612,0.309427,0.250061,0.253838,0.302722,0.206792,0.361081,0.240781,0.326777,0.258597
1,Data\raw\19th Chopin Competition\first round\J...,"Etude in B minor, Op. 25 No. 10.wav",107.666016,708.135774,0.039206,-360.99484,179.01400,8.742428,13.150568,3.218473,...,0.285088,0.312752,0.335405,0.262582,0.333921,0.253349,0.244072,0.199233,0.298447,0.325245
2,Data\raw\19th Chopin Competition\first round\J...,"Etude in C sharp minor, Op. 25 No. 7.wav",103.359375,499.354572,0.016662,-483.60144,174.20236,42.300007,9.218043,11.405186,...,0.261208,0.398315,0.410002,0.249447,0.247719,0.216792,0.376153,0.289132,0.236779,0.331476
3,Data\raw\19th Chopin Competition\first round\J...,"Waltz in E flat major, Op. 18.wav",129.199219,806.788059,0.030811,-367.50998,186.68181,1.799065,3.057348,6.562649,...,0.268085,0.410334,0.214665,0.277850,0.218564,0.320412,0.374215,0.230588,0.400884,0.209436
4,Data\raw\19th Chopin Competition\first round\M...,"Ballade in G minor, Op. 23.wav",123.046875,710.942017,0.026875,-406.55798,170.36528,13.175137,10.507523,8.086515,...,0.396168,0.326942,0.245506,0.259027,0.255346,0.344018,0.242401,0.290132,0.336942,0.207567
5,Data\raw\19th Chopin Competition\first round\M...,"Etude in A minor, Op. 25 No. 11.wav",112.347147,862.591986,0.041351,-318.55070,185.93402,-20.221272,14.148362,8.593420,...,0.325224,0.351527,0.475134,0.374550,0.270732,0.314471,0.289019,0.341706,0.230709,0.276311
6,Data\raw\19th Chopin Competition\first round\M...,"Nocturne in D flat major, Op. 27 No. 2.wav",107.666016,631.090219,0.013653,-483.91623,172.84541,33.885800,8.511955,8.008824,...,0.279875,0.360414,0.266509,0.373952,0.303792,0.196648,0.292168,0.228645,0.203188,0.160956
7,Data\raw\19th Chopin Competition\first round\M...,"Waltz in A flat major, Op. 34 No. 1.wav",117.453835,741.977086,0.025197,-399.13034,176.17268,10.095620,4.442665,10.741858,...,0.278829,0.488884,0.226361,0.261842,0.206429,0.264491,0.358904,0.215798,0.273174,0.171661
8,Data\raw\19th Chopin Competition\first round\P...,"Etude in B minor, Op. 25 No. 10.wav",135.999178,806.915160,0.033248,-372.01462,176.07730,5.033669,10.744130,2.017829,...,0.298258,0.324371,0.361009,0.276428,0.336754,0.283997,0.274721,0.227778,0.324364,0.370634
9,Data\raw\19th Chopin Competition\first round\P...,"Fantasy in F minor, Op. 49.wav",151.999081,615.486592,0.032661,-396.01758,174.53964,17.146347,15.559967,10.493519,...,0.278675,0.368612,0.302609,0.332049,0.301373,0.314879,0.343745,0.270264,0.381785,0.288737
